# Detecting network intrusions & anomalies with higher-order topological kernels via quantum computation notebook

## 0. Install dependencies & hardware backend

0.0. Dependencies

In [1]:
import os
from quask.core_implementation.qiskit_kernel import QiskitKernel
from quask.core import KernelType, Ansatz, KernelFactory
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/pennylane/__init__.py:21: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


0.1. Establish creds if running on IBM hardware (assumes IBM Quantum Cloud account exists and credentials saved with save_account())

In [2]:
# service = QiskitRuntimeService(instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b8ff6077c08a4ea9871560ccb827d457:d3452110-b228-4c79-8959-15ea8cfd435d::") # assuming creds saved with save_account()
# backend = service.backend("ibm_rensselaer")
print("Running on Qiskit simulator (noiseless mode).")

Running on Qiskit simulator (noiseless mode).


In [3]:
from qiskit_aer import Aer

print("✅ qiskit imported successfully!")


✅ qiskit imported successfully!


## 1. Create Kernels

### 1.0. Configure for either quantum simulator or hardware backend

Simulator (noiseless) backend

In [3]:
def create_qiskit_noiseless(ansatz, measurement: str, type: KernelType):
    # Here platform="infty_shots" forces Estimator/Aer path inside QiskitKernel
    return QiskitKernel(
        ansatz,
        measurement,
        type,
        platform="infty_shots",
        n_shots=None  # None means no sampling, use statevector/estimator
    )

# Register the noiseless factory (not needed if already registered)
KernelFactory.add_implementation('qiskit_noiseless', create_qiskit_noiseless)

# # Select implementation for create_kernel calls (names much match; e.g., 'qiskit_noiseless'):
KernelFactory.set_current_implementation('qiskit_noiseless')
print("KernelFactory set to Qiskit noiseless simulator.")

KernelFactory set to Qiskit noiseless simulator.


Hardware (NISQ) backend

Note: only one implementation can be selected (e.g., qiskit_noiseless OR qiskit_ibm)

### 1.1. Ansatz and kernel creation

Config modified from QuASK iris dataset anomaly detection example; see [QuASK: How to optimize a quantum kernel](https://quask.readthedocs.io/en/latest/tutorials_quask/quask_2_optimizers.html) for alternate optimization techniques

In [10]:
# Rebuild ansatz to match dataset feature count
n_features = qX_train.shape[1]

ansatz = Ansatz(n_features=n_features, n_qubits=4, n_operations=n_features)
ansatz.initialize_to_identity()

# Custom topological pattern (adjust to match 6 features)
ansatz.change_operation(0, new_feature=0, new_wires=[0, 1], new_generator="XX", new_bandwidth=3)
ansatz.change_operation(1, new_feature=1, new_wires=[1, 2], new_generator="XY", new_bandwidth=3)
ansatz.change_operation(2, new_feature=2, new_wires=[2, 3], new_generator="XZ", new_bandwidth=3)
ansatz.change_operation(3, new_feature=3, new_wires=[3, 0], new_generator="YY", new_bandwidth=3)
ansatz.change_operation(4, new_feature=4, new_wires=[0, 2], new_generator="YZ", new_bandwidth=3)
ansatz.change_operation(5, new_feature=5, new_wires=[1, 3], new_generator="ZX", new_bandwidth=3)

# Recreate the kernel with updated ansatz
kernel = KernelFactory.create_kernel(ansatz, "ZZZZ", KernelType.FIDELITY)
print(f"Quantum kernel recreated successfully with {n_features} features.")

Quantum kernel recreated successfully with 6 features.


### 1.2. Instantiate machine learning model

In [11]:
model = SVC(kernel='precomputed')
print("SVC model ready (precomputed quantum kernel).")

SVC model ready (precomputed quantum kernel).


## 2. Fit quantum kernels to SVM model and test on BCCC-CIC-CSE-IDS2018

2.0. Load modified datasets (see KERNELSCRIPT.py for dataset cleaning and reduction)

In [12]:
benign_path = "TEST-DATA-TEMP/500-benign.npy"
attack_path = "TEST-DATA-TEMP/500-attack.npy"

qX1 = np.load(benign_path)
qX2 = np.load(attack_path)

attack_name = os.path.basename(attack_path).replace("attack", "").replace(".npy", "").upper()

print(f"Loaded data for {attack_name}: benign {qX1.shape}, attack {qX2.shape}")

Loaded data for 500-: benign (500, 6), attack (500, 6)


2.1. Create testing and training sets

In [13]:
# Select first N=30 samples
# Select smaller sample size for quick quantum kernel testing
qX1 = qX1[:5]
qX2 = qX2[:5]

# First half of new list contains anomaly data (-1); second half is benign (1)
# Label: -1 = benign, +1 = attack
qX = np.vstack([qX1, qX2])
qy = np.array([-1] * len(qX1) + [1] * len(qX2)) 

# Use 0.2-0.3 test size (train on 70% of data, test on 30)
# Split 70/30
qX_train, qX_test, qy_train, qy_test = train_test_split(qX, qy, test_size=0.3, random_state=42)

print("Training and testing sets prepared.")

Training and testing sets prepared.


2.2. Normalize data

In [14]:
samples = np.append(qX_train, qX_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
qX_train = minmax_scale.transform(qX_train)
qX_test = minmax_scale.transform(qX_test)
print("Data normalized to range (-1, 1).")

Data normalized to range (-1, 1).


### 2.3. Build training matrix using quantum kernel

In [15]:
K_train = kernel.build_kernel(qX_train, qX_train, matrix="train")
model.fit(K_train, qy_train)
print("Quantum kernel model trained successfully.")

/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


Quantum kernel model trained successfully.


### 2.4. Predict the labels for the test data

In [16]:
# Predict the labels for the test data
K_test = kernel.build_kernel(qX_test, qX_train, matrix="test")
y_pred = model.predict(K_test)

/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


### 2.5. Calculate and output QML model accuracy

In [17]:
accuracy = np.sum(qy_test == y_pred) / len(qy_test)
print(f"Accuracy for {attack_name} is {accuracy}")

# Optional additional metrics
# from sklearn.metrics import classification_report
# cr = classification_report(qy_test, y_pred)
# print(cr) 

Accuracy for 500- is 0.3333333333333333


## 3. Further notes

The above demo handles a singular network attack, split for improved readability. Below is our testing across all attacks.

In [18]:
data_dir = 'TEST-DATA-TEMP'

for fname in os.listdir(data_dir):
    if fname.endswith('.npy') and 'benign' in fname:
        benign_path = os.path.join(data_dir, fname)
        
        # Construct corresponding attack file name
        attack_fname = fname.replace('benign', 'attack')
        attack_path = os.path.join(data_dir, attack_fname)

        attack_label = attack_fname.replace("attack", "").replace("500", "").replace(".npy", "").upper()

        if os.path.exists(attack_path):
            # Load both arrays
            qX1 = np.load(benign_path)
            qX2 = np.load(attack_path)

            # select first 30 samples
            qX1 = qX1[:30]
            qX2 = qX2[:30]

            # Create testing/training sets
            qX = np.vstack([qX1, qX2])
            qy = np.array([-1] * len(qX1) + [1] * len(qX2))

            qX_train, qX_test, qy_train, qy_test = train_test_split(qX, qy, test_size=0.3, random_state=42)
            
            # normalize data
            samples = np.append(qX_train, qX_test, axis=0)
            minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
            qX_train = minmax_scale.transform(qX_train)
            qX_test = minmax_scale.transform(qX_test)

            # Train
            K_train = kernel.build_kernel(qX_train, qX_train, matrix="train")

            # Fit the ML model
            model.fit(K_train, qy_train)
        
            # Test
            K_test = kernel.build_kernel(qX_test, qX_train, matrix="test")
            y_pred = model.predict(K_test)

            # Calculate accuracy
            accuracy = np.sum(qy_test == y_pred) / len(qy_test)
            print(f"Accuracy for {attack_label} is {accuracy}")


/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/dr.ivanramirez/QML-NIDS/venv/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


Accuracy for - is 0.5
